In [2]:
# ============================================================
# pair_generate_v10_best_bo.py
#  - raw/train.csv → train_month.csv, pivot
#  - base pair mining (lag-corr)
#  - EDA feature 생성
#  - (1) Random Search 로 1차 필터 탐색 (trial 수 늘림)
#  - (2) skopt 기반 Bayesian Optimization (있으면)로 미세 조정
#  - 최종 filter 적용된 pair만 pairs_v10_best.csv로 저장
# ============================================================

from __future__ import annotations
from typing import Dict, Tuple

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LinearRegression

# ------------------------------------------------------------
# (옵션) Bayesian Optimization (scikit-optimize)
# ------------------------------------------------------------
try:
    from skopt import gp_minimize
    from skopt.space import Real
    SKOPT_AVAILABLE = True
except ImportError:
    SKOPT_AVAILABLE = False
    print("[WARN] scikit-optimize(skopt) 미설치 → Bayesian Optimization 건너뜀")


# ============================================================
# PATH
# ============================================================
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

RAW_PATH          = DATA_DIR / "raw" / "train.csv"
PROCESSED_DIR     = DATA_DIR / "processed"
TRAIN_MONTH_PATH  = PROCESSED_DIR / "train_month.csv"

OUT_DIR           = PROCESSED_DIR / "v10_pairs"
OUT_DIR.mkdir(parents=True, exist_ok=True)

PIVOT_PATH        = OUT_DIR / "monthly_pivot_v10.csv"
BEST_PAIR_PATH    = OUT_DIR / "pairs_v10_best.csv"
RS_LOG_PATH       = OUT_DIR / "random_search_log.csv"


# ============================================================
# 0. Pivot + train_month(B안) 생성
# ============================================================
def build_train_month_and_pivot(raw_path: Path):
    df = pd.read_csv(raw_path)

    df["year"] = df["year"].astype(int)
    df["month"] = df["month"].astype(int)
    df["hs4"] = df["hs4"].astype(str).str.zfill(4)
    df["value"] = df["value"].astype(float)

    df["ym"] = pd.to_datetime(
        df["year"].astype(str) + "-" + df["month"].astype(str) + "-01"
    )

    # B안 train_month
    train_month = (
        df.groupby(["item_id", "hs4", "year", "month"], as_index=False)["value"]
          .sum()
    )
    train_month.to_csv(TRAIN_MONTH_PATH, index=False)

    monthly = (
        df.groupby(["item_id", "ym"], as_index=False)["value"]
          .sum()
          .rename(columns={"value": "value_sum"})
    )

    pivot = (
        monthly.pivot_table(index="ym", columns="item_id", values="value_sum")
               .sort_index()
               .fillna(0.0)
    )

    pivot.to_csv(PIVOT_PATH)
    return train_month, monthly, pivot


# ============================================================
# 1. base pair mining (lag-corr)
# ============================================================
def safe_corr(x, y):
    if x.std() == 0 or y.std() == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])


def mine_all_pairs(pivot, max_lag=6, min_nonzero=8):
    values_np = pivot.values
    items = pivot.columns.tolist()
    pairs = []

    for i, leader in enumerate(items):
        A = values_np[:, i].astype(float)
        if np.count_nonzero(A) < min_nonzero:
            continue

        for j, follower in enumerate(items):
            if i == j:
                continue
            B = values_np[:, j].astype(float)
            if np.count_nonzero(B) < min_nonzero:
                continue

            best_corr = 0.0
            best_lag = None

            for lag in range(1, max_lag + 1):
                if len(A) <= lag:
                    break
                c = safe_corr(A[:-lag], B[lag:])
                if abs(c) > abs(best_corr):
                    best_corr = c
                    best_lag = lag

            if best_lag is not None:
                pairs.append(
                    {
                        "leading_item_id": leader,
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr,
                    }
                )

    base_pairs = pd.DataFrame(pairs)
    print(f"[PAIR] base_pairs: {len(base_pairs)}")
    return base_pairs


# ============================================================
# 2. Pure Python DTW (band 제한)
# ============================================================
def dtw_distance(a, b, band=20):
    n, m = len(a), len(b)
    dp = np.full((n + 1, m + 1), np.inf)
    dp[0, 0] = 0.0

    for i in range(1, n + 1):
        j_start = max(1, i - band)
        j_end = min(m, i + band)
        for j in range(j_start, j_end + 1):
            cost = abs(a[i - 1] - b[j - 1])
            dp[i, j] = cost + min(
                dp[i - 1, j],
                dp[i, j - 1],
                dp[i - 1, j - 1],
            )
    return dp[n, m]


# ============================================================
# 3. EDA Feature 생성
# ============================================================
def period_corr(x, y, start, end):
    if end > len(x):
        return np.nan
    xa, ya = x[start:end], y[start:end]
    if xa.std() == 0 or ya.std() == 0:
        return np.nan
    return np.corrcoef(xa, ya)[0, 1]


def trend_slope(series):
    X = np.arange(len(series)).reshape(-1, 1)
    lr = LinearRegression().fit(X, series)
    return lr.coef_[0]


def rolling_corr_std(a, b, lag, window=12):
    xa = a[:-lag]
    ya = b[lag:]
    s = pd.Series(xa).rolling(window).corr(pd.Series(ya))
    return s.std()


def detrended_snr(series):
    x = np.arange(len(series))
    lr = LinearRegression().fit(x.reshape(-1, 1), series)
    trend = lr.predict(x.reshape(-1, 1))
    noise = series - trend
    return trend.var() / (noise.var() + 1e-6)


def series_vol(x):
    return np.std(np.diff(x))


def attach_eda_features(pairs_df, pivot, train_raw):
    hs4_map = (
        train_raw[["item_id", "hs4"]]
        .drop_duplicates()
        .set_index("item_id")["hs4"]
    )

    rows = []
    for _, r in pairs_df.iterrows():
        A = pivot[r.leading_item_id].values
        B = pivot[r.following_item_id].values
        lag = int(r.best_lag)

        corr_e = period_corr(A[:-lag], B[lag:], 0, 24)
        corr_m = period_corr(A[:-lag], B[lag:], 12, 36)
        corr_range = (
            abs(corr_e - corr_m)
            if (not pd.isna(corr_e) and not pd.isna(corr_m))
            else np.nan
        )

        tA = trend_slope(A)
        tB = trend_slope(B)

        rc_std = rolling_corr_std(A, B, lag)
        dtw_d = dtw_distance(A, B)
        snr = detrended_snr(B)
        fv = series_vol(B)

        hs4_lead = hs4_map[r.leading_item_id]
        hs4_foll = hs4_map[r.following_item_id]

        rows.append(
            {
                "corr_early": corr_e,
                "corr_mid": corr_m,
                "corr_range": corr_range,
                "trend_leader": tA,
                "trend_follower": tB,
                "trend_match": np.sign(tA * tB),
                "rollcorr_std": rc_std,
                "dtw_dist": dtw_d,
                "snr_detrend": snr,
                "f_vol": fv,
                "hs4_leader": hs4_lead,
                "hs4_follower": hs4_foll,
                "hs4_equal": int(hs4_lead == hs4_foll),
                "hs4_prefix2_equal": int(str(hs4_lead)[:2] == str(hs4_foll)[:2]),
                "hs4_dist": abs(int(hs4_lead) - int(hs4_foll)),
            }
        )

    eda_df = pd.DataFrame(rows)
    full = pd.concat([pairs_df.reset_index(drop=True), eda_df], axis=1)
    print(f"[EDA] pairs with EDA: {full.shape}")
    return full


# ============================================================
# 4. 필터 스코어 함수
# ============================================================
def apply_filter(df: pd.DataFrame, params: Dict[str, float]) -> pd.DataFrame:
    cond = (
        (df["max_corr"].abs() >= params["corr_min"]) &
        (df["corr_range"] <= params["range_max"]) &
        (df["rollcorr_std"] <= params["roll_max"]) &
        (df["dtw_dist"] <= params["dtw_max"]) &
        (df["f_vol"] <= params["fvol_max"]) &
        (df["snr_detrend"] >= params["snr_min"])
    )
    return df.loc[cond].copy()


def evaluate_filter(df: pd.DataFrame, params: Dict[str, float]) -> Tuple[float, float]:
    filtered = apply_filter(df, params)
    n = len(filtered)

    if n == 0:
        return -1e9, 0.0  # 최악

    # 적당한 pair 수 유도 (대략 800~2000 사이 선호)
    if n < 300:
        count_penalty = 0.5
    elif n > 2500:
        count_penalty = 0.5
    else:
        count_penalty = 0.0

    mean_abs_corr = filtered["max_corr"].abs().mean()
    mean_snr = filtered["snr_detrend"].clip(lower=0).mean()

    # 기본 스코어
    base = 0.6 * mean_abs_corr + 0.3 * mean_snr

    # pair 수 비율
    count_factor = np.clip(n / 1500.0, 0.3, 2.0)

    score = base * count_factor - count_penalty
    return float(score), float(n)


# ============================================================
# 5. Random Search (trial 수 늘린 버전)
# ============================================================
def random_search_filters(df: pd.DataFrame,
                          n_trials: int = 300,
                          random_state: int = 42):

    rng = np.random.RandomState(random_state)
    logs = []

    # 탐색 범위 (EDA보고 잡은 heuristic)
    for t in range(n_trials):
        params = {
            "corr_min": rng.uniform(0.25, 0.45),
            "range_max": rng.uniform(0.25, 0.9),
            "roll_max": rng.uniform(0.30, 0.65),
            "dtw_max": rng.uniform(5e8, 2.5e9),
            "fvol_max": rng.uniform(3e6, 1.5e7),
            "snr_min": rng.uniform(0.03, 0.25),
        }

        score, n = evaluate_filter(df, params)
        logs.append({**params, "pair_count": n, "score": score})

    log_df = pd.DataFrame(logs)
    best_idx = log_df["score"].idxmax()
    best_params = log_df.loc[best_idx].to_dict()

    print(f"\n[RandomSearch] Best trial:")
    print(best_params)

    # 로그 저장
    log_df.to_csv(RS_LOG_PATH, index=False)
    print(f"Random search log saved → {RS_LOG_PATH}")

    # score, pair_count 제거하고 파라미터만 반환
    for k in ["score", "pair_count"]:
        best_params.pop(k, None)

    return best_params, log_df


# ============================================================
# 6. Bayesian Optimization (옵션: skopt)
# ============================================================
def bayes_optimize_filters(df: pd.DataFrame,
                           init_params: Dict[str, float],
                           n_calls: int = 70):

    if not SKOPT_AVAILABLE:
        raise RuntimeError("skopt 미설치")

    # search space 정의
    space = [
        Real(0.25, 0.45, name="corr_min"),
        Real(0.25, 0.9,  name="range_max"),
        Real(0.30, 0.65, name="roll_max"),
        Real(5e8, 2.5e9, name="dtw_max"),
        Real(3e6, 1.5e7, name="fvol_max"),
        Real(0.03, 0.25, name="snr_min"),
    ]

    def objective(x):
        params = {
            "corr_min": x[0],
            "range_max": x[1],
            "roll_max": x[2],
            "dtw_max": x[3],
            "fvol_max": x[4],
            "snr_min": x[5],
        }
        score, _ = evaluate_filter(df, params)
        return -score  # gp_minimize는 최소화 → 마이너스

    # 초기점: random search best
    x0 = [
        init_params["corr_min"],
        init_params["range_max"],
        init_params["roll_max"],
        init_params["dtw_max"],
        init_params["fvol_max"],
        init_params["snr_min"],
    ]

    res = gp_minimize(
        objective,
        space,
        x0=[x0],
        y0=[objective(x0)],
        n_calls=n_calls,
        random_state=0,
    )

    bo_best = {
        "corr_min": res.x[0],
        "range_max": res.x[1],
        "roll_max": res.x[2],
        "dtw_max": res.x[3],
        "fvol_max": res.x[4],
        "snr_min": res.x[5],
    }

    score, n = evaluate_filter(df, bo_best)
    print("\n[BayesOpt] Best params:")
    print(bo_best)
    print(f"[BayesOpt] score={score:.6f}, pair_count={n}")

    return bo_best, res


# ============================================================
# 7. main
# ============================================================
def main():
    # raw / pivot
    train_raw = pd.read_csv(RAW_PATH)
    _, _, pivot = build_train_month_and_pivot(RAW_PATH)

    # base pairs
    base_pairs = mine_all_pairs(pivot)

    # EDA features
    full_pairs = attach_eda_features(base_pairs, pivot, train_raw)

    print("=== Random Search for filters ===")
    rs_best, _ = random_search_filters(full_pairs, n_trials=80, random_state=42)

    # Bayesian Optimization (옵션)
    if SKOPT_AVAILABLE:
        print("\n=== Bayesian Optimization for filters ===")
        final_params, _ = bayes_optimize_filters(full_pairs, init_params=rs_best, n_calls=30)
    else:
        print("\n[INFO] skopt 없음 → Random Search 결과만 사용")
        final_params = rs_best

    print("\n[FINAL PARAMS]")
    print(final_params)

    # 최종 필터 적용
    best_pairs = apply_filter(full_pairs, final_params).reset_index(drop=True)
    print(f"[FINAL] filtered pairs: {len(best_pairs)}")

    # 저장
    best_pairs.to_csv(BEST_PAIR_PATH, index=False)
    print(f"[SAVE] pairs_v10_best.csv → {BEST_PAIR_PATH} (rows={len(best_pairs)})")
    print("✅ DONE")


if __name__ == "__main__":
    main()


[PAIR] base_pairs: 8556
[EDA] pairs with EDA: (8556, 19)
=== Random Search for filters ===

[RandomSearch] Best trial:
{'corr_min': 0.2550701486830915, 'range_max': 0.8757214695406513, 'roll_max': 0.592593042179272, 'dtw_max': 1891948412.187396, 'fvol_max': 7907435.3329712385, 'snr_min': 0.06812475041558608, 'pair_count': 2215.0, 'score': 0.516994748647272}
Random search log saved → /data/ephemeral/home/data/processed/v10_pairs/random_search_log.csv

=== Bayesian Optimization for filters ===

[BayesOpt] Best params:
{'corr_min': 0.2544451341114171, 'range_max': 0.9, 'roll_max': 0.6025480362465228, 'dtw_max': 2430743656.6125507, 'fvol_max': 6867044.62442157, 'snr_min': 0.07007691663109258}
[BayesOpt] score=0.522587, pair_count=2234.0

[FINAL PARAMS]
{'corr_min': 0.2544451341114171, 'range_max': 0.9, 'roll_max': 0.6025480362465228, 'dtw_max': 2430743656.6125507, 'fvol_max': 6867044.62442157, 'snr_min': 0.07007691663109258}
[FINAL] filtered pairs: 2234
[SAVE] pairs_v10_best.csv → /data/ep